# **Tugas 6 RNN Berita Indonesia politik vs olahraga**

In [13]:
import torch
print(torch.__version__)  # Harusnya 2.1.0

from transformers import BertTokenizer, BertForSequenceClassification
print("Transformers OK")


2.6.0+cu124
Transformers OK


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# **Load dataset**

In [3]:
df = pd.read_csv("/kaggle/input/indonesian-news-dataset/data.csv")

# **Hanya ambil kolom 'title' dan 'content'**

In [4]:
df = df[['title', 'content']].dropna()

# **Buat label berdasarkan kata kunci (sederhana)**

In [5]:
def detect_label(text):
    text = text.lower()
    if any(k in text for k in ['liga', 'timnas', 'sepak bola', 'fifa', 'piala', 'olahraga', 'bulu tangkis']):
        return 'olahraga'
    elif any(k in text for k in ['politik', 'presiden', 'dpr', 'partai', 'pemilu', 'menteri']):
        return 'politik'
    else:
        return None

df['label'] = df['content'].apply(detect_label)
df = df[df['label'].isin(['politik', 'olahraga'])].reset_index(drop=True)

# **Label encoding**

In [6]:
df['target'] = df['label'].map({'politik': 0, 'olahraga': 1})

# **Gabungkan teks**

In [7]:
df['text'] = df['title'] + " " + df['content']

# **Split data**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2, random_state=42)


# **Tokenisasi**

In [9]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_len = 200
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# **Bangun model LSTM**

In [10]:
model = Sequential([
    Embedding(10000, 64),
    LSTM(64),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


I0000 00:00:1750093059.894628      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# **Train model**

In [11]:
model.fit(X_train_pad, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5


I0000 00:00:1750093064.336301      95 cuda_dnn.cc:529] Loaded cuDNN version 90300


334/334 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.8496 - loss: 0.4359 - val_accuracy: 0.9113 - val_loss: 0.2984
Epoch 2/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9225 - loss: 0.2588 - val_accuracy: 0.9124 - val_loss: 0.2912
Epoch 3/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9272 - loss: 0.2163 - val_accuracy: 0.9379 - val_loss: 0.2393
Epoch 4/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9662 - loss: 0.1201 - val_accuracy: 0.9349 - val_loss: 0.2262
Epoch 5/5
334/334 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.9747 - loss: 0.0904 - val_accuracy: 0.9397 - val_loss: 0.2231


# **Evaluasi**

In [12]:
y_pred = (model.predict(X_test_pad) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
              precision    recall  f1-score   support

           0       0.96      0.97      0.97      2879
           1       0.81      0.77      0.79       460

    accuracy                           0.94      3339
   macro avg       0.89      0.87      0.88      3339
weighted avg       0.94      0.94      0.94      3339

[[2794   85]
 [ 104  356]]
